In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import models, layers
from tensorflow.keras.optimizers import RMSprop

from tensorflow.keras.applications.vgg16 import VGG16

from typing import List
import os
import numpy as np
import matplotlib.pyplot as plt

In [3]:
model = models.Sequential()

input_shape = (224, 224, 3)

c = 64
model.add(layers.Conv2D(filters=c, 
                        kernel_size=3, 
                        padding='same', 
                        activation='relu',
                        input_shape=input_shape))

# defualt pool_size=(2, 2)
model.add(layers.MaxPool2D(padding='valid'))

c = 128
model.add(layers.Conv2D(filters=c, 
                        kernel_size=3, 
                        padding='same', 
                        activation='relu'))

model.add(layers.MaxPool2D(padding='valid'))

c = 256
model.add(layers.Conv2D(filters=c, 
                        kernel_size=3, 
                        padding='same', 
                        activation='relu'))

model.add(layers.Conv2D(filters=c, 
                        kernel_size=3, 
                        padding='same', 
                        activation='relu'))

model.add(layers.MaxPool2D(padding='valid'))

c = 512
model.add(layers.Conv2D(filters=c, 
                        kernel_size=3, 
                        padding='same', 
                        activation='relu'))

model.add(layers.Conv2D(filters=c, 
                        kernel_size=3, 
                        padding='same', 
                        activation='relu'))

model.add(layers.MaxPool2D(padding='valid'))

c = 512
model.add(layers.Conv2D(filters=c, 
                        kernel_size=3, 
                        padding='same', 
                        activation='relu'))

model.add(layers.Conv2D(filters=c, 
                        kernel_size=3, 
                        padding='same', 
                        activation='relu'))

model.summary()


Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 56, 56, 256)       295168    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 56, 56, 256)       590080    
_____________________________________________

2022-09-20 19:24:49.587003: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-20 19:24:49.587130: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [16]:
# DONE Sequential or Functional
def vggnet_model(config: List, batch_norm: bool, input_shape: List):
    model = models.Sequential()
    c = config[0]

    model.add(layers.Conv2D(filters=c, 
                            kernel_size=3, 
                            padding='same', 
                            activation='relu',
                            input_shape=input_shape))

    for c in config[1:]:
        if c == 'M':
            model.add(layers.MaxPool2D())
        else:
            model.add(layers.Conv2D(filters=c, 
                                    kernel_size=3, 
                                    activation='relu',
                                    padding='same'))

    model.add(layers.Dense(4096, input_dim=(512 * 7 * 7)))
    model.add(layers.Dense(4096))
    model.add(layers.Dense(4096))
    model.add(layers.Dense(1000, activation='softmax'))

    return model

In [17]:
vgg_A_config = [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M']

vgg_B_config = [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 
                512, 512, 'M', 512, 512, 'M']

vgg_C_config = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 
                512, 512, 512, 'M', 512, 512, 512, 'M']

vgg_E_config = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 
                512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M']

In [19]:
input_shape = [224, 224, 3]
test_vggnet = vggnet_model(config=vgg_A_config, batch_norm=True, input_shape=input_shape)

print(test_vggnet.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_48 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_49 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 56, 56, 256)       295168    
_________________________________________________________________
conv2d_51 (Conv2D)           (None, 56, 56, 256)       590080    
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 28, 28, 256)      

In [ ]:
input_shape = [150, 150, 3]
vggnet = vggnet_model(config=vgg_A_config, batch_norm=True, input_shape=input_shape)

In [ ]:
vggnet.compile(loss='binary_crossentropy',
               optimizer=RMSprop(learning_rate=1e-4),
               metrics=['acc'])

In [ ]:
folder_dir = os.getcwd()
original_dataset_dir = folder_dir + '/datasets/cats_and_dogs/train'

base_dir = folder_dir + '/datasets/cats_and_dogs_small'

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150, 150),
                                                    batch_size=20,
                                                    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(validation_dir,
                                                        target_size=(150, 150),
                                                        batch_size=20,
                                                        class_mode='binary')



In [ ]:
for data_batch, labels_batch in train_generator:
    print(f"data batch shape  : {data_batch.shape}")
    print(f"labels batch shape: {labels_batch.shape}")
    break

In [ ]:
history = vggnet.fit_generator(train_generator,
                              steps_per_epoch=100,
                              epochs=30,
                              validation_data=validation_generator,
                              validation_steps=50)
        
vggnet.save('./models/cats_and_dogs_vggnet.h5')

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()